<a href="https://colab.research.google.com/github/arumshin-dev/python_conda_jupyter/blob/main/codeit/3_5_11_RAG_OpenAI_LangChain1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 미션 14: RAG 시스템 구현

## Part 2: OpenAI Embedding & LLM으로 RAG 구축

**LangChain 1.0+ 호환 버전 (LCEL 방식)**

이번 실습에서는 **PyMuPDFLoader**로 문서를 로드하고, **OpenAI**의 Embedding과 LLM을 사용하여 완전한 RAG 시스템을 구축합니다.

### 📋 RAG 파이프라인 개요

```
┌─────────────┐    ┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│  PDF 문서   │ -> │   Chunking  │ -> │  Embedding  │ -> │ Vector Store│
│ PyMuPDFLoader│    │ TextSplitter │    │   OpenAI    │    │    FAISS    │
└─────────────┘    └─────────────┘    └─────────────┘    └─────────────┘
                                                                │
┌─────────────┐    ┌─────────────┐    ┌─────────────┐          │
│    답변     │ <- │     LLM     │ <- │  Retriever  │ <────────┘
│             │    │ GPT-4o-mini │    │   검색기     │
└─────────────┘    └─────────────┘    └─────────────┘
```

---

## 1️⃣ 환경 설정

In [ ]:
# 필요한 라이브러리 설치 (LangChain 1.0+)
!pip install -q pymupdf langchain langchain-openai langchain-community faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
# 버전 확인
import langchain
print(f"LangChain 버전: {langchain.__version__}")

LangChain 버전: 1.1.2


In [ ]:
# OpenAI API 키 설정
import os
from getpass import getpass

# Colab에서 API 키 입력받기
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key를 입력하세요: ")

OpenAI API Key를 입력하세요: ··········


In [ ]:
# 라이브러리 임포트 (LangChain 1.0+ 방식)
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import time



In [ ]:
# # PDF 파일 경로
# PDF_PATH = "2024년_원천징수의무자를_위한_연말정산_신고안내.pdf"

# Google Colab에서 실행시 파일 업로드
from google.colab import files
uploaded = files.upload()

PDF_PATH = list(uploaded.keys())[0]

Saving 2024년_원천징수의무자를_위한_연말정산_신고안내.pdf to 2024년_원천징수의무자를_위한_연말정산_신고안내.pdf


---

## 2️⃣ 문서 로드 (PyMuPDFLoader)

Part 1에서 비교한 결과, **PyMuPDFLoader**가 속도와 한글 문서 처리에서 우수했습니다.

In [ ]:
# PyMuPDFLoader로 문서 로드
print("="*60)
print("📄 PyMuPDFLoader로 문서 로드 중...")
print("="*60)

start_time = time.time()
loader = PyMuPDFLoader(PDF_PATH)
documents = loader.load()
load_time = time.time() - start_time

print(f"\n로드 완료!")
print(f"총 페이지 수: {len(documents)}")
print(f"소요 시간: {load_time:.2f}초")

📄 PyMuPDFLoader로 문서 로드 중...

로드 완료!
총 페이지 수: 426
소요 시간: 1.08초


In [ ]:
# 로드된 문서 샘플 확인
print("📋 첫 번째 페이지 메타데이터:")
print(documents[0].metadata)

print("\n첫 번째 페이지 내용 (처음 500자):")
print(documents[0].page_content[:500])

📋 첫 번째 페이지 메타데이터:
{'producer': 'ezPDF Builder Supreme', 'creator': '', 'creationdate': '2024-12-22T23:44:00+09:00', 'source': '2024년_원천징수의무자를_위한_연말정산_신고안내.pdf', 'file_path': '2024년_원천징수의무자를_위한_연말정산_신고안내.pdf', 'total_pages': 426, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-01-09T17:28:20+09:00', 'trapped': '', 'modDate': "D:20250109172820+09'00'", 'creationDate': "D:20241222234400+09'00'", 'page': 0}

첫 번째 페이지 내용 (처음 500자):
연말정산
신고안내
일 하나는 제대로 하는,
국민께 인정받는 국세청
2024. 12.
2
0
2
4
 
원
천
징
수
의
무
자
를
 
위
한
맞춤형 안내
간소화 서비스
일괄제공 서비스
발
간
등
록
번
호
11-1210000-000072-10


---

## 3️⃣ 텍스트 청킹 (Chunking)

### 왜 Chunking이 필요한가?

1. **LLM 컨텍스트 제한**: LLM에는 최대 입력 토큰 제한이 있음
2. **검색 정확도**: 작은 청크가 더 정확한 검색 결과를 제공
3. **임베딩 품질**: 적절한 크기의 텍스트가 더 의미있는 임베딩 생성

### RecursiveCharacterTextSplitter

- 여러 구분자(`\n\n`, `\n`, ` `, ``)를 순차적으로 시도
- 문장/문단 경계를 최대한 유지하면서 분할
- 한글 문서에도 효과적

In [ ]:
# 텍스트 분할기 설정 (langchain-text-splitters)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # 청크 최대 크기 (문자 수)
    chunk_overlap=200,    # 청크 간 중복 (문맥 유지)
    length_function=len,
    separators=["\n\n", "\n", " ", ""]  # 분할 우선순위
)

print("TextSplitter 설정")
print(f"   - chunk_size: 1000")
print(f"   - chunk_overlap: 200")

TextSplitter 설정
   - chunk_size: 1000
   - chunk_overlap: 200


In [ ]:
# 문서 분할 실행
print("\n" + "="*60)
print("문서 청킹 중...")
print("="*60)

start_time = time.time()
chunks = text_splitter.split_documents(documents)
chunk_time = time.time() - start_time

print(f"\n청킹 완료!")
print(f"원본 문서: {len(documents)}개 → 청크: {len(chunks)}개")
print(f"평균 청크 수/페이지: {len(chunks)/len(documents):.1f}개")
print(f"소요 시간: {chunk_time:.2f}초")


문서 청킹 중...

청킹 완료!
원본 문서: 426개 → 청크: 634개
평균 청크 수/페이지: 1.5개
소요 시간: 0.06초


In [ ]:
# 청크 샘플 확인
print("청크 샘플 (10번째 청크)")
print("-"*50)
sample_chunk = chunks[10]
print(f"메타데이터: {sample_chunk.metadata}")
print(f"\n내용 ({len(sample_chunk.page_content)}자):")
print(sample_chunk.page_content)

청크 샘플 (10번째 청크)
--------------------------------------------------
메타데이터: {'producer': 'ezPDF Builder Supreme', 'creator': '', 'creationdate': '2024-12-22T23:44:00+09:00', 'source': '2024년_원천징수의무자를_위한_연말정산_신고안내.pdf', 'file_path': '2024년_원천징수의무자를_위한_연말정산_신고안내.pdf', 'total_pages': 426, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-01-09T17:28:20+09:00', 'trapped': '', 'modDate': "D:20250109172820+09'00'", 'creationDate': "D:20241222234400+09'00'", 'page': 5}

내용 (191자):
등록된 자를 위해 지출한 의료비, 난임시술비는 한도 없음
※ 산후조리원비용 : 출산 1회당 200만원 한도
※ 장애인활동지원급여* 비용 중 실제 지출한 본인부담금
* 「장애인활동 지원에 관한 법률」에 따라 수급자에게 제공되는 활동보조, 방문목욕, 방문간호 등 서비스
과세표준
산출세액
(-) ‌세액감면 및 
세액공제
(x) 기본세율
146P


In [ ]:
# 청크 크기 분포 확인
chunk_sizes = [len(chunk.page_content) for chunk in chunks]

print("청크 크기 통계")
print(f"   - 최소: {min(chunk_sizes)}자")
print(f"   - 최대: {max(chunk_sizes)}자")
print(f"   - 평균: {sum(chunk_sizes)/len(chunk_sizes):.0f}자")

청크 크기 통계
   - 최소: 3자
   - 최대: 1000자
   - 평균: 681자


---

## 4️⃣ OpenAI Embedding

### OpenAI Embedding 모델

| 모델 | 차원 | 특징 |
|------|------|------|
| `text-embedding-3-small` | 1536 | 빠르고 저렴, 대부분의 용도에 적합 |
| `text-embedding-3-large` | 3072 | 더 정확, 복잡한 검색에 유리 |


In [ ]:
# OpenAI Embedding 모델 초기화
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"  # 비용 효율적인 모델 선택
)

print("OpenAI Embedding 모델 초기화 완료")
print(f"   - 모델: text-embedding-3-small")
print(f"   - 차원: 1536")

OpenAI Embedding 모델 초기화 완료
   - 모델: text-embedding-3-small
   - 차원: 1536


In [ ]:
# Embedding 테스트
test_text = "연말정산 의료비 공제 한도는 얼마인가요?"
test_embedding = embeddings.embed_query(test_text)

print("Embedding 테스트")
print(f"   - 입력 텍스트: '{test_text}'")
print(f"   - 벡터 차원: {len(test_embedding)}")
print(f"   - 벡터 샘플 (처음 5개): {test_embedding[:5]}")

Embedding 테스트
   - 입력 텍스트: '연말정산 의료비 공제 한도는 얼마인가요?'
   - 벡터 차원: 1536
   - 벡터 샘플 (처음 5개): [0.005206627305597067, -0.053752630949020386, 0.01164640299975872, 0.025548309087753296, -0.024578653275966644]


---

## 5️⃣ Vector Store (FAISS)

### FAISS:

- **Facebook AI Similarity Search**
- 대규모 벡터 유사도 검색을 위한 라이브러리
- 로컬에서 실행 가능 (서버 불필요)
- 빠른 검색 속도

### 다른 Vector Store 옵션

| Vector Store | 특징 |
|--------------|------|
| **FAISS** | 로컬, 빠름, 설치 간단 |
| **Chroma** | 로컬, 영구 저장, 메타데이터 필터링 |
| **Pinecone** | 클라우드, 확장성, 관리형 |

In [ ]:
# FAISS Vector Store 생성
print("="*60)
print("FAISS Vector Store 생성 중...")
print("="*60)
print(f"\n임베딩할 청크 수: {len(chunks)}개")

start_time = time.time()
vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)
embed_time = time.time() - start_time

print(f"\nVector Store 생성 완료!")
print(f"소요 시간: {embed_time:.2f}초")
print(f"처리 속도: {len(chunks)/embed_time:.1f} 청크/초")

FAISS Vector Store 생성 중...

임베딩할 청크 수: 634개

Vector Store 생성 완료!
소요 시간: 8.85초
처리 속도: 71.6 청크/초


---

## 6️⃣ 유사도 검색 테스트

RAG 체인을 구성하기 전에, 검색이 잘 작동하는지 테스트해봅니다.

In [ ]:
# 유사도 검색 테스트
query = "의료비 세액공제 한도"

print(f"검색 쿼리: '{query}'")
print("="*60)

# 상위 3개 문서 검색
similar_docs = vectorstore.similarity_search(query, k=3)

for i, doc in enumerate(similar_docs, 1):
    print(f"\n결과 {i} (페이지: {doc.metadata.get('page', 'N/A')})")
    print("-"*50)
    print(doc.page_content[:400] + "...")

검색 쿼리: '의료비 세액공제 한도'

결과 1 (페이지: 231)
--------------------------------------------------
공제대상
체험학습비
기타 자료
50만원
공제대상(30만원)
수능응시료
국세청 자료
20만원
공제대상
학원비
-
120만원
공제대상 아님
이강모
대학원 수강료(비과세학자금)
-
300만원
공제대상 아님
＊교육비 세액공제 ⇨ 630,000원
  
구분
공제대상
공제대상 제외
공제 한도
세액공제 대상금액
세액공제액
취학전아동
120만원
300만원
120만원
630,000
고등학생
335만원
120만원
300만원
300만원
근로자 본인
300만원
없음
-
합계
455만원
420만원
420만원
  ※ 세액공제액 계산：세액공제 대상금액(4,200,000원) × 15% = 630,000원...

결과 2 (페이지: 231)
--------------------------------------------------
원천징수의무자를 위한 
2024년 연말정산 신고안내
216
○ 의료비 세액공제
＊의료비 세액공제 ⇨ 950,700원
  
구분
수술
입원치료비
시력교정용안경
산후조리원 비용
세액공제
대상금액
세액공제액
본인,
난임시술비
230만원*
55만원 →
50만원한도
280.0만원
950,700원
그 외
부양가족
250만원
220만원 →
한도 200만원
253.8만원
합계
250만원
230만원
50만원
200만원
533.8만원
＊입원치료비 130만원, 난임시술비 100만원
국세청자료
250만원
230만원
200만원
기타 자료
50만원
  ＊시력교정용 안경을 제외하고 나머지는 국세청자료에 해당
    - 세액공제 대상금액 계산：5,338,000원 (②+③+④)
      ① 총급여액의 3% → 1,962,000원 (65...

결과 3 (페이지: 4)
--------------------------------------------------
총급여액의 25%를 초과하는 금액의 15%~40%를 소득공제
* ‌공제한도 :

In [ ]:
# 유사도 점수와 함께 검색
query = "연장근로수당 계산 방법"

print(f"검색 쿼리: '{query}'")
print("="*60)

# 유사도 점수 포함 검색
docs_with_scores = vectorstore.similarity_search_with_score(query, k=3)

for i, (doc, score) in enumerate(docs_with_scores, 1):
    print(f"\n결과 {i}")
    print(f"   유사도 점수: {score:.4f} (낮을수록 유사)")
    print(f"   페이지: {doc.metadata.get('page', 'N/A')}")
    print(f"   내용: {doc.page_content[:200]}...")

검색 쿼리: '연장근로수당 계산 방법'

결과 1
   유사도 점수: 1.1142 (낮을수록 유사)
   페이지: 417
   내용: 총액에서 ｢근로기준법｣에 따른 연장근로·야간근로 또는 휴일근로를 
하여 통상임금에 더하여 받는 급여 및 ｢선원법｣에 따라 받는 생산수당
(비율급으로 받는 경우에는 월 고정급을 초과하는 비율급을 말한다)을 
뺀 급여를 말함
소령 §17
23
일용근로자
근로를 제공한 날 또는 시간에 따라 근로대가를 계산하거나 근로를 
제공한 날 또는 시간의 근로성과에 따라 급...

결과 2
   유사도 점수: 1.1147 (낮을수록 유사)
   페이지: 221
   내용: ③국외근로소득금액(근로소득금액에 포함된 국외근로소득)계산
(18,000,000원－6,000,000원) －⎛
⎝9,000,000원 × 12,000,000원⎞
⎠＝7,680,000원
25,000,000원
    ④외국납부세액 공제한도액 계산
690,000원 ×
7,680,000원
＝ 331,200원
16,000,000원
        ※ 근로소득 산출세액 계산...

결과 3
   유사도 점수: 1.1201 (낮을수록 유사)
   페이지: 387
   내용: 원천징수의무자를 위한 
2024년 연말정산 신고안내
372
(이용방법) ① 홈택스(www.hometax.go.kr) →My홈택스→연말정산→지급명세서 등 제출내역 또는  
② 홈택스(www.hometax.go.kr) →지급명세서·자료제출·공익법인→(근로·사업 등) 
    지급명세서 제출→근로소득 지급명세서 조회(근로자용)
다. 과거 연금보험료 등 소득·세액...


---

## 7️⃣ OpenAI LLM 설정

### GPT 모델 선택 가이드

| 모델 | 특징 | 비용 (1M tokens) |
|------|------|------------------|
| `gpt-4o` | 최신, 가장 강력 | $2.50 / $10 |
| `gpt-4o-mini` | 빠르고 저렴, 대부분 충분 | $0.15 / $0.60 |
| `gpt-4-turbo` | 긴 컨텍스트 (128K) | $10 / $30 |

In [ ]:
# OpenAI LLM 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,          # 일관된 답변을 위해 0으로 설정
    max_tokens=1000         # 최대 출력 토큰
)

print("OpenAI LLM 초기화 완료")
print(f"   - 모델: gpt-4o-mini")
print(f"   - temperature: 0")
print(f"   - max_tokens: 1000")

OpenAI LLM 초기화 완료
   - 모델: gpt-4o-mini
   - temperature: 0
   - max_tokens: 1000


In [ ]:
# LLM 테스트
response = llm.invoke("안녕하세요, 간단히 자기소개 해주세요.")
print("LLM 테스트 응답:")
print(response.content)

LLM 테스트 응답:
안녕하세요! 저는 AI 언어 모델로, 다양한 주제에 대해 정보를 제공하고 질문에 답변하는 역할을 하고 있습니다. 여러분의 궁금증을 해결하고, 필요한 정보를 찾는 데 도움을 드리기 위해 여기 있습니다. 어떤 질문이든 편하게 해주세요!


---

## 8️⃣ RAG 체인 구성 (LCEL 방식)

### LangChain Expression Language (LCEL)

LangChain 1.0에서는 **LCEL**을 사용하여 체인을 구성합니다.

```python
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
```

In [ ]:
# Retriever 생성
retriever = vectorstore.as_retriever(
    search_type="similarity",   # 유사도 검색
    search_kwargs={"k": 4}      # 상위 4개 문서 검색
)

print("Retriever 생성 완료")
print(f"   - 검색 타입: similarity")
print(f"   - 검색 개수: 4")

Retriever 생성 완료
   - 검색 타입: similarity
   - 검색 개수: 4


In [ ]:
# 프롬프트 템플릿
template = """당신은 한국 세법 전문가입니다.
주어진 문서 내용을 바탕으로 질문에 정확하게 답변해주세요.

문서에 답변이 없는 경우, "제공된 문서에서 해당 정보를 찾을 수 없습니다"라고 답변하세요.
추측하지 말고, 문서에 있는 내용만 답변하세요.

참고 문서:
{context}

질문: {question}

답변:"""

prompt = ChatPromptTemplate.from_template(template)

print("프롬프트 템플릿 설정 완료")

프롬프트 템플릿 설정 완료


In [ ]:
# 문서 포맷팅 함수
def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    return "\n\n".join(
        f"[페이지 {doc.metadata.get('page', 'N/A')}]\n{doc.page_content}"
        for doc in docs
    )

print("문서 포맷팅 함수 정의 완료")

문서 포맷팅 함수 정의 완료


In [ ]:
# LCEL 방식으로 RAG 체인 구성
rag_chain = (
    {
        "context": retriever | format_docs,   # 검색 → 포맷팅
        "question": RunnablePassthrough()     # 질문 그대로 전달
    }
    | prompt                                  # 프롬프트 적용
    | llm                                     # LLM 호출
    | StrOutputParser()                       # 문자열로 파싱
)

print("RAG 체인 구성 완료! (LCEL 방식)")
print("\n 체인 구성:")
print("   Query → Retriever → format_docs → Prompt → LLM → Output")

RAG 체인 구성 완료! (LCEL 방식)

 체인 구성:
   Query → Retriever → format_docs → Prompt → LLM → Output


---

## 9️⃣ RAG 시스템 테스트

이제 완성된 RAG 시스템으로 연말정산 관련 질문을 테스트해봅니다.

In [ ]:
def ask_question(question):
    """RAG 시스템에 질문하고 결과를 출력합니다."""
    print("="*70)
    print(f"질문: {question}")
    print("="*70)

    start_time = time.time()

    # LCEL 체인 실행
    answer = rag_chain.invoke(question)

    response_time = time.time() - start_time

    print(f"\n답변:")
    print(answer)

    # 참조 문서 별도 조회
    source_docs = retriever.invoke(question)
    print(f"\n참조 문서 ({len(source_docs)}개):")
    for i, doc in enumerate(source_docs, 1):
        page = doc.metadata.get('page', 'N/A')
        print(f"   {i}. 페이지 {page}: {doc.page_content[:80]}...")

    print(f"\n응답 시간: {response_time:.2f}초")
    print("\n")

    return answer

In [ ]:
# 테스트 질문 1: 의료비 공제
result1 = ask_question("의료비 세액공제 한도는 얼마인가요?")

질문: 의료비 세액공제 한도는 얼마인가요?

답변:
제공된 문서에서 해당 정보를 찾을 수 없습니다.

참조 문서 (4개):
   1. 페이지 231: 공제대상
체험학습비
기타 자료
50만원
공제대상(30만원)
수능응시료
국세청 자료
20만원
공제대상
학원비
-
120만원
공제대상 아님
이강모
...
   2. 페이지 230: Ⅰ. 2024년 귀속 연말정산 종합사례
215
사. 세액공제 및 세액감면
○ 근로소득세액공제
＊근로소득세액공제 ⇨ 660,000원 = MIN(①...
   3. 페이지 4: 총급여액의 25%를 초과하는 금액의 15%~40%를 소득공제
* ‌공제한도 : 총급여 7천만원 이하는 300만원, 총급여 7천만원 초과는 25...
   4. 페이지 231: 원천징수의무자를 위한 
2024년 연말정산 신고안내
216
○ 의료비 세액공제
＊의료비 세액공제 ⇨ 950,700원
  
구분
수술
입원치료비
...

응답 시간: 1.73초




In [ ]:
# 테스트 질문 2: 연장근로수당
result2 = ask_question("연장근로수당은 어떻게 계산하나요?")

질문: 연장근로수당은 어떻게 계산하나요?

답변:
제공된 문서에서 해당 정보를 찾을 수 없습니다.

참조 문서 (4개):
   1. 페이지 423: 원천징수의무자를 위한 
2024년 연말정산 신고안내
408
근로소득·사업소득·연금소득 연말정산 비교
구분
근로소득
사업소득
연금소득
1. 연말정...
   2. 페이지 417: 총액에서 ｢근로기준법｣에 따른 연장근로·야간근로 또는 휴일근로를 
하여 통상임금에 더하여 받는 급여 및 ｢선원법｣에 따라 받는 생산수당
(비율급...
   3. 페이지 153: 원천징수의무자를 위한 
2024년 연말정산 신고안내
138
○우리사주조합은 우리사주조합원의 출연금 중 소득공제의 대상이 되는 금액과 그러하지 아...
   4. 페이지 88: Ⅰ. 근로소득
73
○｢국군포로의 송환 및 대우 등에 관한 법률｣에 따른 국군포로가 지급받는 보수 및 퇴직일시금
○｢교육기본법｣ 제28조 제1항...

응답 시간: 2.97초




In [ ]:
# 테스트 질문 3: 기본공제
result3 = ask_question("기본공제 대상자의 소득요건은 무엇인가요?")

질문: 기본공제 대상자의 소득요건은 무엇인가요?

답변:
기본공제 대상자의 소득요건은 연간 소득금액이 100만원 이하(근로소득만 있는 자는 총급여액 500만원 이하)이어야 합니다.

참조 문서 (4개):
   1. 페이지 5: 70%(청년 90%) 감면(연간 200만원 한도)
  ‌근로소득세액공제 : 산출세액 130만원 이하분 55%, 초과분 30% 공제(74만원, ...
   2. 페이지 53: ※ 기타소득금액 300만원 이하인 사람은 종합소득 신고 여부를 선택할 수 있으며, 이 경우 종합소득 신고하지 않은 부양가족은 
기본공제 받을 수...
   3. 페이지 3: 4,500만원 초과 1억원 이하
1,200만원 + 4,500만원 초과액의 5% 
총급여액 ×   5% + 975만원
1억원 초과
1,475만원 ...
   4. 페이지 231: 공제대상
체험학습비
기타 자료
50만원
공제대상(30만원)
수능응시료
국세청 자료
20만원
공제대상
학원비
-
120만원
공제대상 아님
이강모
...

응답 시간: 2.25초




In [ ]:
# 테스트 질문 4: 식사대 비과세
result4 = ask_question("식사대 비과세 요건을 알려주세요.")

질문: 식사대 비과세 요건을 알려주세요.

답변:
식사대 비과세 요건은 다음과 같습니다:

1) 현물 식사
   - 근로자가 사내급식 또는 이와 유사한 방법으로 제공받는 식사 기타 음식물은 비과세한다. 
   - 비과세되는 식사·기타 음식물은 사용자가 무상으로 제공하는 음식물로서 다음의 요건에 해당하여야 한다:
     - 통상적으로 급여에 포함되지 아니함
     - 음식물의 제공 여부로 급여에 차등이 없음
     - 사용자가 추가 부담하여 제공

2) 월 20만원 이하 식사대
   - 식사·기타 음식물을 제공받지 아니하는 근로자가 받는 월 20만원 이하 식사대는 비과세되는 근로소득에 해당된다. 

또한, 회사로부터 음식물을 제공받지 아니하고 식사대로 매월 25만원을 지급받는 경우, 20만원은 비과세하고 5만원은 과세된다.

참조 문서 (4개):
   1. 페이지 84: ○식사·기타 음식물을 제공받고 있는 근로자가 별도로 식사대를 지급받는 경우에는 제공받은식사·
기타 음식물에 한하여 비과세되는 급여로 본다. 다만...
   2. 페이지 82: ① 기본급 140만원
⑤ 야간근로수당 10만원
    ② 가족수당(매월 지급) 6만원
⑥ 휴일근로수당 5만원
    ③ 상여 100만원
⑦ 식대...
   3. 페이지 83: 원천징수의무자를 위한 
2024년 연말정산 신고안내
68
   
참고
통상임금에 가산하여 지급받는 급여의 범위
근로기준법 제56조【연장·야간 및...
   4. 페이지 84: Ⅰ. 근로소득
69
   - 통상적으로 급여에 포함되지 아니함
   - 음식물의 제공 여부로 급여에 차등이 없음
   - 사용자가 추가 부담하여...

응답 시간: 7.25초




---

## 🔟 스트리밍 출력 (LCEL 장점)

LCEL의 장점 중 하나는 **스트리밍 출력**을 쉽게 구현할 수 있다는 것입니다.

In [ ]:
# 스트리밍 출력 테스트
question = "연말정산 시 교육비 공제 대상은 무엇인가요?"

print(f"질문: {question}")
print("="*60)
print("\n답변 (스트리밍):")

# stream() 메서드로 스트리밍 출력
for chunk in rag_chain.stream(question):
    print(chunk, end="", flush=True)

print("\n")

질문: 연말정산 시 교육비 공제 대상은 무엇인가요?

답변 (스트리밍):
연말정산 시 교육비 공제 대상은 다음과 같습니다:

1. 근로자 본인을 위하여 지급한 교육비:
   - ｢유아교육법｣, ｢초·중등교육법｣, ｢고등교육법｣ 및 ｢특별법｣에 따른 학교(국외교육기관 포함)
   - ｢평생교육법｣에 따른 고등학교졸업 이하의 학력이 인정되는 학교형태의 평생 교육시설, 전공대학, 원격대학

2. 기본공제대상자인 배우자·직계비속·형제자매 및 입양자를 위하여 지급한 교육비:
   - 초등학교 취학 전 아동, 초·중·고등학생: 1명당 연 300만원 한도 적용
   - 대학생: 1명당 연 900만원 한도 적용
   - 대학원생: 공제대상 아님

3. 장애인 특수교육비(직계존속 포함): 전액

4. 기본공제대상자인 장애인의 재활교육을 위하여 지급하는 비용:
   - 사회복지시설 및 비영리법인에 지출한 비용 전액
   - 발달재활서비스제공기관에 지출한 비용 전액

단, 교육비 세액공제대상에서 제외되는 항목도 있으며, 이는 소득세 또는 증여세가 비과세되는 교육비 등입니다.

